<h1>Power System Analysis and Optimization (5XWA0)</h1>
<h2>Optimal linear (DC) power flow</h2>

<b>MSc Irena Dukovska</b> <br />
<b>Dr. Nikolaos Paterakis</b> <br />
  Electrical Energy Systems Group <br />
  Department of Electrical Engineering <br />
  Eindhoven University of Technology <br />
  (i.dukovska@tue.nl, n.paterakis@tue.nl) <br /><br />


<h2>Introduction</h2>

In this tutorial we will be developing a model in Python in order to solve and examine the solution of a simple form of the Optimal Power Flow (OPF) problem using a linear representation of the network constraints (DC power flow).

The optimization problem is formulated as follows:

\begin{align}
&\text{Minimize} \ C = \sum_{i}(a_i+b_i \cdot P_i + c_i \cdot P_i^2) \\
&\text{subject to:} \\
&P_i^{min} \leq P_i \leq P_i^{max} \ \forall i \in I\\
&F_{l} = B_{l,n} (\delta_{n}-\delta_{k}) \ \forall (n,k) \in L\\
&-F_{l}^{max} \leq F_{l} \leq F_{l}^{max} \ \forall l \in L \\
& \sum_{i \in I^n} P_{i} + \sum_{l | n = \text{Receiving End}}F_{l} = \sum_{j \in J^n} D_{j} +\sum_{l | n = \text{Sending End} }F_{l} \ \forall n \in N \\
&\delta_{n} = 0 \ \text{at reference (slack) bus} \\
&P_i \geq 0 \ \forall i \in I
\end{align}


<ul>
<b>Nomenclature</b>
<li>$i$ is the index of generators ($P_{i}$ in MW)</li>
<li>$j$ is the index of loads ($D_j$ in MW)</li>
<li>$n$ is the index of buses (set $N$)</li>
<li>$l$ is the index of lines (set $L$). The bus pair $(n,k)\in L$ indicates the connection of which buses constitutes a transmission line </li>
<li>$I^n, J^n$ are the sets of generators and loads that are connected to bus $n$, respectively</li>
<li>$F_l$ is the power that flows through line $l$ in MW</li>
<li>$\delta_{n}$ is the voltage angle at bus $n$ in radians</li>
<li>$B_{l,n}$ is the line to node incidence matrix that takes the following values: $1/X_l$ if $n$ is the sending bus of line $l$, $-1/X_l$ if $n$ is the receiving bus of line $l$, and, 0 otherwise. $X_l$ is in per-unit on a common system base.</li>
</ul>

A set of $|N|$ nodal power balance equations has now replaced the single system power balance constraint of the Economic Dispatch problem. In other words, the net supply and demand at each bus must be balanced. This is illustrated in the following figure:

![Illustration of nodal power balance constraints](img/balanceOPF.png)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *

def load_data(file_name):
    UnitData = pd.read_excel(file_name, index_col = 0, sheet_name='Units')
    LoadData = pd.read_excel(file_name, index_col = 0, sheet_name='Load')
    NetworkData = pd.read_excel(file_name, index_col = 0, sheet_name='Network')
    return UnitData, LoadData, NetworkData

def optimization_model(op_type, UnitData, LoadData, NetworkData):

    '''
    Accepted op_type (optimization problem type) are:
    'ED': solves a simple economic dispatch
    'OPF_wotl': solves optimal power flow without transmission line limits
    'OPF_wtl': solves optimal power flow with transmission line limits
    '''

    model = ConcreteModel()

    #import suffixes (marginal values) -- "import them from the solver"
    model.dual = Suffix(direction=Suffix.IMPORT)

    # Define Sets
    model.I = Set(ordered=True, initialize=UnitData.index)        #set of generators
    model.J = Set(ordered=True, initialize=LoadData.index)        #set of loads
    model.L = Set(ordered=True, initialize=NetworkData.index)     #set of lines
    buses = NetworkData['FROM'].append(NetworkData['TO']).unique()
    model.N = Set(ordered=True, initialize=buses)                 #set of nodes

    # Generator
    model.a = Param(model.I, within=NonNegativeReals, mutable=True)
    model.b = Param(model.I, within=NonNegativeReals, mutable=True)
    model.c = Param(model.I, within=NonNegativeReals, mutable=True)
    model.Pmax = Param(model.I, within=NonNegativeReals, mutable=True)
    model.Pmin = Param(model.I, within=NonNegativeReals, mutable=True)
    # Loads
    model.D = Param(model.J, within=NonNegativeReals, mutable=True)
    # Transmission lines
    model.P_line_max = Param(model.L, within=NonNegativeReals, mutable=True)
    model.X = Param(model.L, within=NonNegativeReals, mutable=True)
    # Power flow parameters
    model.Bmat = Param(model.L, model.N, within = Reals, mutable=True)

    for i in model.I:
        model.a[i] = UnitData.loc[i,'a']
        model.b[i] = UnitData.loc[i,'b']
        model.c[i] = UnitData.loc[i,'c']
        model.Pmax[i] = UnitData.loc[i,'Max']
        model.Pmin[i] = UnitData.loc[i,'Min']

    for j in model.J:
        model.D[j] = LoadData.loc[j,'MW']

    for l in model.L:
        model.P_line_max[l] = NetworkData.loc[l, 'P_line_max']
        model.X[l] = NetworkData.loc[l, 'X']

    for l in model.L:
        for n in model.N:
            if NetworkData.loc[l,'FROM'] == n:
                model.Bmat[l,n] = 1/model.X[l]
            elif NetworkData.loc[l,'TO'] == n:
                model.Bmat[l,n] = -1/model.X[l]
            else:
                 model.Bmat[l,n] = 0

    # Define Decision Variables
    model.P = Var(model.I, within=NonNegativeReals)
    model.theta = Var(model.N, within=Reals)
    model.F = Var(model.L, within = Reals)

    #Define constraints
    def cost_rule(model):
        return sum(model.a[i] + model.b[i]*model.P[i]+model.c[i]*model.P[i]*model.P[i] for i in model.I)

    def minmax_rule(model, i):
        # DEPRECEATED model.Pmin[i] <= model.P[i] <= model.Pmax[i]
        return inequality(model.Pmin[i], model.P[i], model.Pmax[i])

    def angle_ini_rule(model):
        return model.theta['Bus1'] == 0

    def flow_limit_rule(model,l):
        return inequality(-model.P_line_max[l], model.F[l], model.P_line_max[l])

    # create flow for line l between nodes n1 and n2
    def line_flow_rule(model, l, n1, n2):
        if ((n1 == NetworkData.loc[l,'FROM']) and (n2 == NetworkData.loc[l,'TO'])) \
            or ((n1 == NetworkData.loc[l,'TO']) and (n2 == NetworkData.loc[l,'FROM'])):
            return model.F[l] == model.Bmat[l,n1] * (model.theta[n1]-model.theta[n2])
        else:
            return Constraint.Skip #If the pair of buses does not constitute a line, then do not create a constraint

    def pf_rule(model, n):
        return sum(model.P[i] for i in model.I if UnitData.loc[i,'Location'] == n) + \
               sum(model.F[l] for l in model.L if NetworkData.loc[l,'TO'] == n) == \
               sum(model.D[j] for j in model.J if LoadData.loc[j,'Location'] == n) + \
               sum(model.F[l] for l in model.L if NetworkData.loc[l,'FROM'] == n)

    def pbalance_rule(model):
        return sum(model.P[i] for i in model.I) == sum(model.D[j] for j in model.J)

    if op_type == 'ED':
        model.cost = Objective(rule = cost_rule)
        model.unit_out_constraints = Constraint(model.I, rule = minmax_rule)
        model.pfrule = Constraint(model.N, rule = pf_rule)

        opt=SolverFactory('gurobi')
        opt.options["MIPGap"] = 0.0
        results=opt.solve(model)

        return model

    if op_type == 'OPF_wotl':
        model.cost = Objective(rule = cost_rule)
        model.unit_out_constraints = Constraint(model.I, rule = minmax_rule)
        model.angleini = Constraint(rule = angle_ini_rule)
        model.lfrule = Constraint(model.L,model.N,model.N,rule=line_flow_rule)
        model.pfrule = Constraint(model.N, rule = pf_rule)

        opt=SolverFactory('gurobi')
        opt.options["MIPGap"] = 0.0
        results=opt.solve(model)

        return model

    if op_type == 'OPW_wtl':
        model.cost = Objective(rule = cost_rule)
        model.unit_out_constraints = Constraint(model.I, rule = minmax_rule)
        model.angleini = Constraint(rule = angle_ini_rule)
        model.flowlim = Constraint(model.L, rule = flow_limit_rule)
        model.lfrule = Constraint(model.L,model.N,model.N,rule=line_flow_rule)
        model.pfrule = Constraint(model.N, rule = pf_rule)

        opt=SolverFactory('gurobi')
        opt.options["MIPGap"] = 0.0
        results=opt.solve(model)

        return model

    else:
        print('UNKNOWN OPTIMIZATION PROBLEM')

data_file_2bus = 'OPF_input_simple_two_bus.xlsx'
data_file_6bus = 'OPF_input_six_bus.xlsx'

UnitData, LoadData, NetworkData = load_data(data_file_2bus)
model = optimization_model('ED', UnitData, LoadData, NetworkData)

print('Total cost (Euro): ', model.cost())
print('Nodal marginal cost (Euro/MWh): ', [(n,model.pfrule[n].get_suffix_value(model.dual)) for n in model.N])
print('Power output (MW):', [(i,model.P[i].value) for i in model.I])


Total cost (Euro):  4189.000000000001
Nodal marginal cost (Euro/MWh):  [('Bus1', 7.779999999999102), ('Bus2', 7.779999999999102)]
Power output (MW): [('Unit1', 310.00000000000836), ('Unit2', 189.99999999999167)]
